In [6]:
# @title Setup Environment
import os

!sudo apt-get update && sudo apt-get install -y ffmpeg

if not os.path.exists("heartlib"):
    !git clone https://github.com/HeartMuLa/heartlib.git

%cd heartlib

!pip install -e .
!pip install -u "huggingface_hub[cli]"
!pip install flash-attn --no-build-isolation
!pip install accelerate

Hit:1 https://cli.github.com/packages stable InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:4 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:7 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading packag

In [7]:
# @title Download Checkpoints
import os

if os.path.basename(os.getcwd()) != "heartlib":
    %cd heartlib

os.makedirs('./ckpt', exist_ok=True)

!huggingface-cli download --local-dir './ckpt' 'HeartMuLa/HeartMuLaGen'
!huggingface-cli download --local-dir './ckpt/HeartMuLa-oss-3B' 'HeartMuLa/HeartMuLa-oss-3B'
!huggingface-cli download --local-dir './ckpt/HeartCodec-oss' 'HeartMuLa/HeartCodec-oss'
!huggingface-cli download --local-dir './ckpt/HeartTranscriptor-oss' 'HeartMuLa/HeartTranscriptor-oss'

⚠️  Warning: 'huggingface-cli download' is deprecated. Use 'hf download' instead.
Fetching 4 files:   0% 0/4 [00:00<?, ?it/s]Downloading 'README.md' to 'ckpt/.cache/huggingface/download/Xn7B-BWUGOee2Y6hCZtEhtFu4BE=.b2034b283373042c5a09732470eae847e70c88a0.incomplete'

README.md: 1.89kB [00:00, 10.2MB/s]
Download complete. Moving file to ckpt/README.md

tokenizer.json: 0.00B [00:00, ?B/s]Downloading '.gitattributes' to 'ckpt/.cache/huggingface/download/wPaCkH-WbT7GsmxMKKrNZTV4nSM=.a6344aac8c09253b3b630fb776ae94478aa0275b.incomplete'


.gitattributes: 1.52kB [00:00, 6.80MB/s]
Download complete. Moving file to ckpt/.gitattributes
Fetching 4 files:  25% 1/4 [00:00<00:00,  3.32it/s]Downloading 'gen_config.json' to 'ckpt/.cache/huggingface/download/EpjBFd09FpMaURtnmYyhqAvQM-s=.3a5413112a5917ce99497eb3ba9ac97072fb30f2.incomplete'


gen_config.json: 100% 101/101 [00:00<00:00, 677kB/s]
Download complete. Moving file to ckpt/gen_config.json
tokenizer.json: 9.09MB [00:00, 161MB/s]
Download comple

In [8]:
# @title Run Music Generation
# No Touchy
import os
from IPython.display import Audio, display
import torch

torch.backends.cuda.matmul.allow_tf32 = True
torch.backends.cudnn.allow_tf32 = True
torch.set_float32_matmul_precision('high')

os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'max_split_size_mb:512'
os.environ['CUDA_LAUNCH_BLOCKING'] = '0'

# -----------------------------------------------------------------------------
# 1. SETUP YOUR LYRICS AND TAGS (you can touch this lol)
# -----------------------------------------------------------------------------
my_lyrics = """
[Verse]
The sun creeps in across the floor
I hear the traffic outside the door
The coffee pot begins to hiss
It is another morning just like this

[Chorus]
Every day the light returns
Every day the fire burns
"""

my_tags = "piano,happy,pop"
# -----------------------------------------------------------------------------
# Do Not Touch This Code Below, lol.

if os.path.basename(os.getcwd()) != "heartlib":
    if os.path.exists("heartlib"):
        %cd heartlib
    else:
        raise FileNotFoundError("Repo not found. Please run Block 1 (Setup) first.")

!python ./examples/run_music_generation.py \
    --model_path=./ckpt \
    --version="3B" \
    --lyrics="$my_lyrics" \
    --tags="$my_tags" \
    --save_path="./assets/output.mp3"

if os.path.exists("./assets/output.mp3"):
    print("✓ Generation complete!")
    display(Audio("./assets/output.mp3"))
else:
    print("✗ Generation failed. Check logs above.")


2026-01-19 20:31:10.883978: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2026-01-19 20:31:10.901572: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1768854670.922731   13691 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1768854670.929242   13691 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1768854670.945807   13691 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 